In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os

from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.training_week import generate_training_week

In [5]:
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df(strava_client)

athlete_id='98390356' token expired at 2024-08-05 21:51:07.862388+00:00, refreshing token...


No rates present in response headers


In [6]:
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)

In [7]:
training_week = generate_training_week(weekly_summaries, day_of_week_summaries)

mon: rest day, 0.0 miles, Complete rest day to allow for recovery and muscle repair.
tues: moderate run, 7.0 miles, Moderate pace; focus on maintaining steady pace and proper form.
wed: speed workout, 6.0 miles, Include interval training: 5x800m at 5K pace with 400m rest intervals.
thurs: moderate run, 7.0 miles, Moderate pace; incorporate some rolling hills if possible.
fri: easy run, 5.0 miles, Relaxed pace; keep it easy to prepare for the long run ahead.
sat: long run, 14.0 miles, Long slow distance; maintain a steady, conversational pace.
sun: easy run, 3.0 miles, Short recovery run; keep the pace easy to aid recovery.

In [12]:
from src.email_manager import send_email
from src.types.training_week import TrainingWeek

def format_training_week_to_html(training_week: TrainingWeek) -> str:
    html_content = """
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                color: #333;
                margin: 0;
                padding: 0;
            }
            .container {
                width: 100%;
                max-width: 600px;
                margin: 20px auto;
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                overflow: hidden;
            }
            .header {
                background-color: #4CAF50;
                color: #ffffff;
                text-align: center;
                padding: 20px;
            }
            .header h1 {
                margin: 0;
                font-size: 24px;
            }
            .content {
                padding: 20px;
            }
            .content h2 {
                color: #4CAF50;
                font-size: 20px;
                margin-bottom: 10px;
            }
            .content ul {
                list-style-type: none;
                padding: 0;
                margin: 0;
            }
            .content li {
                background-color: #f9f9f9;
                margin-bottom: 10px;
                padding: 15px;
                border-left: 5px solid #4CAF50;
                border-radius: 5px;
            }
            .content li strong {
                display: block;
                font-size: 16px;
                margin-bottom: 5px;
            }
            .footer {
                background-color: #f1f1f1;
                text-align: center;
                padding: 10px;
                font-size: 12px;
                color: #777;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>Your Training Schedule</h1>
            </div>
            <div class="content">
                <h2>Get pumped for this week's training.</h2>
                <ul>
    """
    for day, session in training_week.dict().items():
        html_content += f"""
                    <li>
                        <strong>{day.capitalize()}</strong>
                        <span>{session['session_type'].value}</span><br>
                        <span>Distance: {session['distance']} miles</span><br>
                        <span>Notes: {session['notes']}</span>
                    </li>
        """
    html_content += """
                </ul>
            </div>
            <div class="footer">
                <p>Powered by the Strava API and OpenAI</p>
            </div>
        </div>
    </body>
    </html>
    """
    return html_content


html_content = format_training_week_to_html(training_week)
send_email(
    subject="This Week's Training Schedule Just Dropped 🏃‍♂️",
    html_content=html_content,
)

{'message_id': '<202408052216.57922177966@smtp-relay.mailin.fr>',
 'message_ids': None}